# 目的說明



#去yahoo抓取當下股票情況 價錢/漲跌

【2024-07-27】

#可以Run Real time or daily 

#Real Time : 可將昨日推薦股的情況，搭配當日股價去做line推撥   (等比較穩定點，之後再做)

#Daily : 可在每日收盤後執行，來複盤 

In [1]:
import pandas as pd
import re
import time
from datetime import datetime
import random
import requests
from bs4 import BeautifulSoup
%run _BaseInfo.ipynb
%run _LineMsg.ipynb
%run _CrawBase.ipynb
#【產出】資料結果放這裡
file_name = 'DailyResultWeb.xlsx'
# 備份
current_date = datetime.now().strftime("%Y-%m-%d") #動態時間For資料夾

log_directory='log/RunRealTimeStock'
_bk_file_name = f"{log_directory}/{current_date}{file_name}" 


In [2]:

def yahoo_twstock_crawler(stock):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                           AppleWebKit/537.36 (KHTML, like Gecko) \
                           Chrome/102.0.0.0 Safari/537.36'}
    url=f'https://tw.stock.yahoo.com/quote/{stock}' 
    print(url)
    r=requests.get(url, headers=headers)
    soup=BeautifulSoup(r.text, 'lxml')
    # 存储匹配的标签
    tag = []

    # 选择器1
    tag0=soup.select('span[class*="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c)"]')
    if tag0:
        tag.append(tag0[0])  # 只添加非空标签的第一个元素

    # 选择器2
    tag1=soup.select('span[class*="Fz(32px) Fw(b) Lh(1) Mend(16px) C(#fff) Px(6px) Py(2px) Bdrs(4px) "]')
    
    if tag1:
        tag.append(tag1[0])  # 只添加非空标签的第一个元素
    # Fz(32px) Fw(b) Lh(1) Mend(16px) C(#fff) Px(6px) Py(2px) Bdrs(4px) Bgc($c-trend-down)
    # 选择器3
    tag2=soup.select('span[class*="Fz(32px) Fw(b) Lh(1) Mend(16px) C(#fff) Px(6px) Py(2px) Bdrs(4px) Bgc($c-trend-up)"]')
    if tag2:
        tag.append(tag2[0])  # 只添加非空标签的第一个元素

    # 打印结果
    #print("tag0:", tag0)
    #print("tag1:", tag1)
    #print("tag2:", tag2)
    #print("tag:", tag)
    if tag:
        price=float(tag[0].text.replace(',',''))
        
        # 擷取漲跌價
        tag=soup.select('span[class*="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c)"]')
        if 'C($c-trend-up)' in tag[0].attrs['class']:   
            change_price='+' + tag[0].text    
        elif 'C($c-trend-down)' in tag[0].attrs['class']:   
            change_price='-' + tag[0].text   
        else:    
            change_price=tag[0].text
        # 擷取漲跌幅
        tag=soup.select('span[class*="Jc(fe) Fz(20px) Lh(1.2) Fw(b) D(f) Ai(c)"]')
        change_quote=tag[0].text[1:-1]
        if 'C($c-trend-up)' in tag[0].attrs['class']:   
            change_quote=f'(+{change_quote})'   
        elif 'C($c-trend-down)' in tag[0].attrs['class']:   
            change_quote=f'(-{change_quote})'   
        else:   
            change_quote=f'({change_quote})'

        return price, change_price, change_quote
    return '-', '-', '-'

In [9]:
stocks=pd.read_excel('check.xlsx' ).dropna()['stock_number'].drop_duplicates()
alert_msg_list=['注意! 盤中漲跌幅超過 2% :']
df= stocks.rename('stock_number').to_frame("stock_number")

for stock in stocks:
    try:
        print(stock)
        price, change_price, change_quote=yahoo_twstock_crawler(stock)
        name=GetStockInfoByID(str(stock)).split('-')[0]
        alert_msg=f'{stock} {name} {price} : {change_quote}'
        #print(alert_msg)
        df.loc[df['stock_number'] == stock, 'name']=name
        df.loc[df['stock_number'] == stock, 'now_price'] = price
        df.loc[df['stock_number'] == stock, 'change_price'] = change_price
        df.loc[df['stock_number'] == stock, 'change_quote'] = change_quote.replace('(','').replace(')','')
        df.loc[df['stock_number'] == stock, 'change_quote_']=float(re.search(r'-?\d+\.\d+', change_quote).group())
        change_quote2=float(re.search(r'\d+\.\d+', change_quote).group())
        
        if change_quote2 > 2 or change_quote2 < -2:
            alert_msg_list.append(alert_msg)
        time.sleep(1)
    except:
        print('error'+stock)
print('\n'.join(str(x) for x in alert_msg_list))
#sendLineMessage('\n'.join(str(x) for x in alert_msg_list))

6117
https://tw.stock.yahoo.com/quote/6117
4966
https://tw.stock.yahoo.com/quote/4966
4953
https://tw.stock.yahoo.com/quote/4953
4564
https://tw.stock.yahoo.com/quote/4564
2402
https://tw.stock.yahoo.com/quote/2402
2376
https://tw.stock.yahoo.com/quote/2376
2367
https://tw.stock.yahoo.com/quote/2367
2349
https://tw.stock.yahoo.com/quote/2349
2301
https://tw.stock.yahoo.com/quote/2301
1599
https://tw.stock.yahoo.com/quote/1599
1101
https://tw.stock.yahoo.com/quote/1101
注意! 盤中漲跌幅超過 2% :
4564 元翎 22.3 : (+2.06%)
2376 技嘉 282.5 : (-2.92%)
2349 錸德 15.35 : (+2.68%)
2301 光寶科 101.5 : (-3.33%)
1101 台泥 33.15 : (+2.47%)


In [10]:
df[df['change_quote_']>2]

,stock_number,name,now_price,change_price,change_quote,change_quote_
30,4564,元翎,22.30,+0.45,+2.06%,2.06
70,2349,錸德,15.35,+0.40,+2.68%,2.68
100,1101,台泥,33.15,+0.80,+2.47%,2.47


In [5]:
alert_msg_list

['注意! 盤中漲跌幅超過 2% :',
 '4564 元翎 22.3 : (+2.06%)',
 '2376 技嘉 282.5 : (-2.92%)',
 '2349 錸德 15.35 : (+2.68%)',
 '2301 光寶科 101.5 : (-3.33%)',
 '1101 台泥 33.15 : (+2.47%)']